<a href="https://colab.research.google.com/github/deepapaikar/NLP/blob/main/T5_for_Abstractive_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installations

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.6 MB/s eta 0:00:00


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00


In [ ]:
!pip install SentencePiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.4 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.5 MB/s eta 0:00:00


In [ ]:
!pip install transformers[torch]

## Import necessary libraries

In [ ]:
from transformers import pipeline, T5ForConditionalGeneration, T5Tokenizer, TrainingArguments, Trainer, DataCollatorForSeq2Seq
import pandas as pd
from datasets import Dataset
import random

In [ ]:
# Step 1: Load the pre-trained T5 model
# 't5-small' refers to the name of the pre-trained model, which is a smaller version of T5.
# The 'from_pretrained' function loads the pre-trained weights and architecture.
base_model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Step 2: Load the corresponding tokenizer for the T5 model
# The tokenizer is used to convert text inputs into numerical tokens, which the model can understand.
# 't5-small' refers to the tokenizer that matches the pre-trained model we loaded in Step 1.
base_tokenizer = T5Tokenizer.from_pretrained('t5-small')

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [ ]:
# Import the necessary library for mounting Google Drive
from google.colab import drive

# Mount Google Drive to access files from the Google Drive storage
drive.mount('/content/drive')

Mounted at /content/gdrive


In [ ]:
# Read the CSV file containing Amazon Fine Food Reviews
reviews = pd.read_csv('/content/drive/MyDrive/Amazon Fine Food Reviews/Reviews.csv')

In [ ]:
# Display the first few rows of the DataFrame 'reviews'
reviews.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
#Pre-processing step
#Punctuation is important in grammar and important for complex decoding architectures to know when to stop!

def add_punc(s):
  # Check if the last character of the string is not a period, question mark, or an empty string
  if s[-1] not in ('.', '', '?'):
    # If the last character is not punctuation, add a period to the end of the string
    s=s+ '.'
  return s

In [ ]:
#Drop rows with missing values (NaN) from the DataFrame 'reviews'
reviews.dropna(inplace=True)
reviews['Summary'] = reviews['Summary'].map(add_punc)
print(reviews.shape)
reviews.head()

(568411, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food.,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised.,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all.",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine.,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy.,Great taffy at a great price. There was a wid...


In [ ]:
# Filter the DataFrame 'reviews' based on the length of the 'Summary' column
# We use boolean indexing to select rows where the length of the 'Summary' string is less than 100 characters and greater than or equal to 30 characters.
reviews = reviews[(reviews['Summary'].str.len() < 100) & (reviews['Summary'].str.len() >= 30)]

In [ ]:
# Get the shape of the DataFrame 'reviews'
reviews.shape

(160205, 10)

In [ ]:
# Set a random seed for reproducibility
# The 'random.seed()' function is used to set the random seed value to ensure reproducibility of results.
random.seed(0)

# Convert the 'reviews' DataFrame to a Hugging Face Dataset and take a random sample of 5000 rows
# The 'astype(str)' method is used to ensure all data in the DataFrame is treated as strings.
# The 'sample(5000)' method randomly selects 5000 rows from the DataFrame to create the 'reviews_dataset'.
reviews_dataset = Dataset.from_pandas(reviews.astype(str).sample(5000))

In [ ]:
# We have a prompt, which is the prefix to be used in the encoder
prefix = "summarize: "

# We will manually add our own labels because unlike GPT, we cannot assume the labels are based on the inputs
# Define a preprocess function to prepare the data for training the T5 model
def preprocess_function(examples):
    # Prepare the inputs for the encoder by adding the 'prefix' to each document in the 'Text' column
    inputs = [prefix + doc for doc in examples["Text"]]
    # Tokenize and encode the 'inputs' using the base tokenizer with a maximum sequence length of 1024 and truncation if needed
    model_inputs = base_tokenizer(inputs, max_length=1024, truncation=True)

    # Tokenize and encode the 'Summary' column to obtain the labels for the decoder
    labels = base_tokenizer(examples["Summary"], max_length=128, truncation=True)
    # Set the 'input_ids' of the labels as the 'labels' for the model_inputs dictionary
    model_inputs["labels"] = labels["input_ids"]

    # Return the model_inputs dictionary with inputs and labels
    return model_inputs


In [ ]:
# Tokenize and preprocess the 'reviews_dataset' using the 'preprocess_function'
tokenized_reviews_dataset = reviews_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
# Split the tokenized_reviews_dataset into training and testing sets
tokenized_reviews_dataset = tokenized_reviews_dataset.train_test_split(test_size=0.1)

In [ ]:
# Data Collator specifically for generic sequence to sequence tasks
# translating one sequence to another like translation, summerization, etc
data_collator = DataCollatorForSeq2Seq(tokenizer=base_tokenizer, model=base_model)

In [ ]:
# Define the training arguments for the T5 model
# The 'TrainingArguments' class is used to set various hyperparameters and configurations for the training process.
training_args = TrainingArguments(
    output_dir="./t5_summary_results",           # Directory to store the model checkpoints and results.
    evaluation_strategy="epoch",                 # Evaluate the model after each epoch during training.
    per_device_train_batch_size=8,               # Batch size for training.
    per_device_eval_batch_size=8,
    num_train_epochs=10,                         # Total number of training epochs.
    load_best_model_at_end=True,
    logging_steps=50,                            # Log training metrics after every 50 steps.
    save_strategy='epoch'                        # Save the model after each epoch.
)

# Initialize the Trainer for training and evaluation
# It takes the model, training arguments, training dataset, evaluation dataset, tokenizer, and data collator as inputs.
trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=tokenized_reviews_dataset["train"],
    eval_dataset=tokenized_reviews_dataset["test"],
    tokenizer=base_tokenizer,
    data_collator=data_collator
)

# Evaluate the model on the evaluation dataset
trainer.evaluate()



{'eval_loss': 3.108182668685913,
 'eval_runtime': 5.3628,
 'eval_samples_per_second': 93.235,
 'eval_steps_per_second': 11.748}

In [ ]:
# Train the T5 model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.210800,3.066964
2,3.164000,3.044823
3,3.019600,3.040526
4,3.045700,3.026039
5,3.062000,3.019951
6,2.960000,3.016089
7,2.825300,3.013635
8,2.917000,3.013211
9,2.791400,3.011506
10,2.867900,3.011136


TrainOutput(global_step=5630, training_loss=2.9967912284349674, metrics={'train_runtime': 1453.4669, 'train_samples_per_second': 30.96, 'train_steps_per_second': 3.873, 'total_flos': 5031954253479936.0, 'train_loss': 2.9967912284349674, 'epoch': 10.0})

In [ ]:
# Evaluate the T5 model on the evaluation dataset
trainer.evaluate()

{'eval_loss': 3.011136293411255,
 'eval_runtime': 4.3127,
 'eval_samples_per_second': 115.936,
 'eval_steps_per_second': 14.608,
 'epoch': 10.0}

In [ ]:
# Save the trained T5 model
trainer.save_model()

In [ ]:
# Load the trained T5 model from the specified directory
loaded_model = T5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/Amazon Fine Food Reviews/t5_summary_results')

# Summarization pipeline prepends a default prefix of summarize
generator = pipeline(
    'summarization', model=loaded_model, tokenizer=base_tokenizer
)

In [ ]:
# Take a random sample of one review from the 'reviews' DataFrame
sam = reviews.sample(1)

# Print the 'Summary' of the randomly selected review
print(sam['Summary'])

# Extract the text from the selected review
text = sam['Text'].tolist()[0]
text

70853    Wonderful Product - Lots of Energy.
Name: Summary, dtype: object


'This is a great product, great value.  I take bottles per day, it gives me a nice boost of energy,  like to take it first thing in the morning and at 3PM or after lunch.  Great taste.'

In [ ]:
# Generate Summary
generator(text, min_length=3, max_length=15, early_stopping=True, num_beams=2)

[{'summary_text': 'Great product, great value.'}]

In [ ]:
# Trying the base T5 on the same text
base_generator = pipeline(
    'summarization', model='t5-base', tokenizer='t5-base'
)

# Summary is a bit more extractive than our fine tuned version and style isn't quite the same as our dataset
base_generator(text, min_length=3, max_length=15, early_stopping=True, num_beams=2)

[{'summary_text': 'i take bottles per day, it gives me a nice boost'}]

In [ ]:
# Trying the T5-small on the same text
base_generator = pipeline(
    'summarization', model='t5-small', tokenizer='t5-base'
)

# Summary is a bit more extractive than our fine tuned version and style isn't quite the same as our dataset
base_generator(text, min_length=3, max_length=15, early_stopping=True, num_beams=2)

[{'summary_text': 'this is a great product, great value . I take bottles'}]

**I am able to see difference in all of these . My mfine tuned model has shorter , crispier summary as compared to base models**